In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import torch.cuda.amp as amp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import kagglehub

path = kagglehub.dataset_download("msambare/fer2013")
print("Path to dataset files:", path)

train_dir = "/kaggle/input/fer2013/train"
val_dir = "/kaggle/input/fer2013/test"

train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # FER2013 is grayscale!
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")
print(f"Classes: {train_dataset.classes}")

model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)

model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.last_channel, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.4),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.3),
    nn.Linear(256, 7)
)

for param in model.features.parameters():
    param.requires_grad = False

model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.classifier.parameters(), lr=3e-4, weight_decay=0.01)

scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-4,
    epochs=10,
    steps_per_epoch=len(train_loader),
    pct_start=0.2
)

scaler = amp.GradScaler()

print("\n" + "="*60)
print("STAGE 1: Training Classifier (Features Frozen)")
print("="*60)

num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total
    avg_loss = running_loss / len(train_loader)


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            with amp.autocast():
                outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} Train Acc: {train_acc:.4f} Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "emotion_model_best.pth")
        print(f"  ✓ Best model saved! Val Acc: {val_acc:.4f}")

print(f"\nStage 1 Best Validation Accuracy: {best_val_acc:.4f}")

print("\n" + "="*60)
print("STAGE 2: Fine-tuning Last Layers")
print("="*60)

for name, param in model.features.named_parameters():

    parts = name.split('.')
    try:
        layer_num = int(parts[0]) if parts[0].isdigit() else int(parts[1]) if len(parts) > 1 and parts[1].isdigit() else -1
    except:
        layer_num = -1


    if layer_num >= 14:
        param.requires_grad = True
        print(f"Unfreezing: {name}")


trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTrainable parameters: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.1f}%)")

optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=7, eta_min=1e-7)

num_epochs_ft = 7

for epoch in range(num_epochs_ft):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total
    avg_loss = running_loss / len(train_loader)


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            with amp.autocast():
                outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = correct / total
    print(f"[Fine-Tune] Epoch [{epoch+1}/{num_epochs_ft}] Loss: {avg_loss:.4f} Train Acc: {train_acc:.4f} Val Acc: {val_acc:.4f}")

    scheduler.step()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "emotion_model_best.pth")
        print(f"  ✓ New best model! Val Acc: {val_acc:.4f}")

print("\n" + "="*60)
print(f"🎯 FINAL Best Validation Accuracy: {best_val_acc:.4f}")
print("="*60)

torch.save(model.state_dict(), "emotion_model_final.pth")
print("\n✓ Training complete! Models saved.")
print("  - emotion_model_best.pth (best validation accuracy)")
print("  - emotion_model_final.pth (final model)")

Using Colab cache for faster access to the 'fer2013' dataset.
Path to dataset files: /kaggle/input/fer2013
Training samples: 28709, Validation samples: 7178
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

STAGE 1: Training Classifier (Features Frozen)


/tmp/ipython-input-3844083860.py:83: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
/tmp/ipython-input-3844083860.py:104: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/tmp/ipython-input-3844083860.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Epoch [1/10] Loss: 2.0372 Train Acc: 0.1925 Val Acc: 0.3050
  ✓ Best model saved! Val Acc: 0.3050
Epoch [2/10] Loss: 1.8283 Train Acc: 0.3112 Val Acc: 0.3658
  ✓ Best model saved! Val Acc: 0.3658
Epoch [3/10] Loss: 1.7426 Train Acc: 0.3408 Val Acc: 0.3763
  ✓ Best model saved! Val Acc: 0.3763
Epoch [4/10] Loss: 1.7065 Train Acc: 0.3553 Val Acc: 0.3823
  ✓ Best model saved! Val Acc: 0.3823
Epoch [5/10] Loss: 1.6887 Train Acc: 0.3656 Val Acc: 0.3970
  ✓ Best model saved! Val Acc: 0.3970
Epoch [6/10] Loss: 1.6800 Train Acc: 0.3677 Val Acc: 0.3887
Epoch [7/10] Loss: 1.6683 Train Acc: 0.3741 Val Acc: 0.3947
Epoch [8/10] Loss: 1.6695 Train Acc: 0.3762 Val Acc: 0.3940
Epoch [9/10] Loss: 1.6600 Train Acc: 0.3788 Val Acc: 0.3959
Epoch [10/10] Loss: 1.6626 Train Acc: 0.3772 Val Acc: 0.3997
  ✓ Best model saved! Val Acc: 0.3997

Stage 1 Best Validation Accuracy: 0.3997

STAGE 2: Fine-tuning Last Layers
Unfreezing: 14.conv.0.0.weight
Unfreezing: 14.conv.0.1.weight
Unfreezing: 14.conv.0.1.bias
Unfr

/tmp/ipython-input-3844083860.py:186: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/tmp/ipython-input-3844083860.py:209: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


[Fine-Tune] Epoch [1/7] Loss: 1.6044 Train Acc: 0.4141 Val Acc: 0.4543
  ✓ New best model! Val Acc: 0.4543
[Fine-Tune] Epoch [2/7] Loss: 1.5293 Train Acc: 0.4558 Val Acc: 0.4812
  ✓ New best model! Val Acc: 0.4812
[Fine-Tune] Epoch [3/7] Loss: 1.4894 Train Acc: 0.4781 Val Acc: 0.4953
  ✓ New best model! Val Acc: 0.4953
[Fine-Tune] Epoch [4/7] Loss: 1.4642 Train Acc: 0.4928 Val Acc: 0.5078
  ✓ New best model! Val Acc: 0.5078
[Fine-Tune] Epoch [5/7] Loss: 1.4443 Train Acc: 0.5055 Val Acc: 0.5127
  ✓ New best model! Val Acc: 0.5127
[Fine-Tune] Epoch [6/7] Loss: 1.4416 Train Acc: 0.5089 Val Acc: 0.5194
  ✓ New best model! Val Acc: 0.5194
[Fine-Tune] Epoch [7/7] Loss: 1.4308 Train Acc: 0.5109 Val Acc: 0.5155

🎯 FINAL Best Validation Accuracy: 0.5194

✓ Training complete! Models saved.
  - emotion_model_best.pth (best validation accuracy)
  - emotion_model_final.pth (final model)


In [21]:
print("\n" + "="*60)
print("STAGE 3: Full Fine-Tuning (Layers 10–17)")
print("="*60)

for name, param in model.features.named_parameters():
    parts = name.split('.')
    try:
        layer_num = int(parts[0]) if parts[0].isdigit() else int(parts[1]) if len(parts) > 1 and parts[1].isdigit() else -1
    except:
        layer_num = -1

    if layer_num >= 10:
        param.requires_grad = True
        print(f"Unfreezing: {name}")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTrainable parameters: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.1f}%)")

from collections import Counter

class_counts_dict = Counter(train_dataset.targets)
num_classes = len(train_dataset.classes)
class_counts = [class_counts_dict.get(i, 1) for i in range(num_classes)]  # avoid zero division
total_samples = sum(class_counts)
class_weights = [total_samples / c for c in class_counts]
class_weights = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)

optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-7)

num_epochs_stage3 = 5
print(f"\nTraining for {num_epochs_stage3} epochs...\n")

for epoch in range(num_epochs_stage3):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total
    avg_loss = running_loss / len(train_loader)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            with torch.cuda.amp.autocast():
                outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = correct / total
    print(f"[Full Fine-Tune] Epoch [{epoch+1}/{num_epochs_stage3}] Loss: {avg_loss:.4f} Train Acc: {train_acc:.4f} Val Acc: {val_acc:.4f}")

    scheduler.step()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "emotion_model_best.pth")
        print(f"  ✓ New best model! Val Acc: {val_acc:.4f}")

print("\n" + "="*60)
print(f"🏁 FINAL Best Validation Accuracy After Stage 3: {best_val_acc:.4f}")
print("="*60)

torch.save(model.state_dict(), "emotion_model_final_stage3.pth")
print("\n✓ Stage 3 complete! Model saved as emotion_model_final_stage3.pth")



STAGE 3: Full Fine-Tuning (Layers 10–17)
Unfreezing: 10.conv.0.0.weight
Unfreezing: 10.conv.0.1.weight
Unfreezing: 10.conv.0.1.bias
Unfreezing: 10.conv.1.0.weight
Unfreezing: 10.conv.1.1.weight
Unfreezing: 10.conv.1.1.bias
Unfreezing: 10.conv.2.weight
Unfreezing: 10.conv.3.weight
Unfreezing: 10.conv.3.bias
Unfreezing: 11.conv.0.0.weight
Unfreezing: 11.conv.0.1.weight
Unfreezing: 11.conv.0.1.bias
Unfreezing: 11.conv.1.0.weight
Unfreezing: 11.conv.1.1.weight
Unfreezing: 11.conv.1.1.bias
Unfreezing: 11.conv.2.weight
Unfreezing: 11.conv.3.weight
Unfreezing: 11.conv.3.bias
Unfreezing: 12.conv.0.0.weight
Unfreezing: 12.conv.0.1.weight
Unfreezing: 12.conv.0.1.bias
Unfreezing: 12.conv.1.0.weight
Unfreezing: 12.conv.1.1.weight
Unfreezing: 12.conv.1.1.bias
Unfreezing: 12.conv.2.weight
Unfreezing: 12.conv.3.weight
Unfreezing: 12.conv.3.bias
Unfreezing: 13.conv.0.0.weight
Unfreezing: 13.conv.0.1.weight
Unfreezing: 13.conv.0.1.bias
Unfreezing: 13.conv.1.0.weight
Unfreezing: 13.conv.1.1.weight
Unfr

/tmp/ipython-input-750139784.py:57: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/tmp/ipython-input-750139784.py:80: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


[Full Fine-Tune] Epoch [1/5] Loss: 1.6213 Train Acc: 0.5179 Val Acc: 0.5358
  ✓ New best model! Val Acc: 0.5358
[Full Fine-Tune] Epoch [2/5] Loss: 1.5213 Train Acc: 0.5435 Val Acc: 0.5649
  ✓ New best model! Val Acc: 0.5649
[Full Fine-Tune] Epoch [3/5] Loss: 1.4518 Train Acc: 0.5690 Val Acc: 0.5743
  ✓ New best model! Val Acc: 0.5743
[Full Fine-Tune] Epoch [4/5] Loss: 1.4084 Train Acc: 0.5740 Val Acc: 0.5943
  ✓ New best model! Val Acc: 0.5943
[Full Fine-Tune] Epoch [5/5] Loss: 1.3843 Train Acc: 0.5865 Val Acc: 0.5964
  ✓ New best model! Val Acc: 0.5964

🏁 FINAL Best Validation Accuracy After Stage 3: 0.5964

✓ Stage 3 complete! Model saved as emotion_model_final_stage3.pth


In [33]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import torch.amp as amp
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.mobilenet_v2(weights=None)


num_classes = 7

model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.last_channel, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.4),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)

model.load_state_dict(torch.load("emotion_model_final_stage3.pth", map_location=device))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3), # FER2013 is grayscale
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def predict_emotion(image_path, class_names):
    """
    Loads an image, preprocesses it, and predicts the emotion.

    Args:
        image_path (str): The path to the input image.
        class_names (list): A list of class names in the correct order.

    Returns:
        tuple: A tuple containing the predicted emotion (str) and confidence (float).
    """
    try:

        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)


        with torch.no_grad():
            with amp.autocast(device_type=device.type):
                outputs = model(input_tensor)
                probs = nn.Softmax(dim=1)(outputs)
                confidence, pred_class_idx = torch.max(probs, dim=1)

        return class_names[pred_class_idx.item()], confidence.item()

    except FileNotFoundError:
        return f"Error: Image file not found at {image_path}", 0.0
    except Exception as e:
        return f"An error occurred: {e}", 0.0


class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
try:
    Image.new('RGB', (100, 100)).save("test_image.jpg")
    print("Created a dummy 'test_image.jpg'. Replace it with a real face image for an actual prediction.")
except:
    pass

image_path = "/content/Screenshot 2025-10-15 162801.png"

emotion, confidence = predict_emotion(image_path, class_names)

print("\n" + "="*40)
print(f"Predicted Emotion: {emotion}")
print(f"Confidence: {confidence:.4f}")
print("="*40)

Created a dummy 'test_image.jpg'. Replace it with a real face image for an actual prediction.

Predicted Emotion: angry
Confidence: 0.6339
